Oceanography python bootcamp, Winter 2025
# Week 6 notebook

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import scipy.interpolate as sint
from scipy.optimize import curve_fit

pd.set_option("mode.copy_on_write", True)

In [ ]:
import week6_magic as magic

## One-dimensional interpolation and curve fitting

### Univariate interpolation

In [ ]:
# loading test data
x_samples = magic.interp_1d_samp_x.copy()
y_samples = magic.interp_1d_samp_y.copy()

In [ ]:
# plot the test data

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlim(0, 10)
ax.set_ylim(0, 1.2)

ax.set_title("Test data for 1D interpolation")

ax.scatter(x_samples, y_samples, c = "black")

plt.show()

In [ ]:
# make linear (k = 1) and cubic (k = 3) interpolations
lin_interp = sint.InterpolatedUnivariateSpline(x_samples, y_samples, k=1)
cubic_interp = sint.InterpolatedUnivariateSpline(x_samples, y_samples, k=3)

In [ ]:
# evaluate the interpolated function on plotting grid
x_range = np.linspace(0, 10, 101)
y_linear = lin_interp(x_range)
y_cubic = cubic_interp(x_range)

In [ ]:
# plot the test data and the interpolations

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlim(0, 10)
ax.set_ylim(0, 1.2)

ax.set_title("Test data and 1D interpolations")

ax.plot(x_range, y_linear, ls=":", label="linear")
ax.plot(x_range, y_cubic, ls="--", label="cubic")
ax.scatter(x_samples, y_samples, c="black", label="data")

ax.legend()

plt.show()

In [ ]:
# evaluating the "ground truth" function at the plot points
y_truth = magic.interp_1d_truth(x_range)

# plot the test data, interpolations, and ground truth

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlim(0, 10)
ax.set_ylim(0, 1.2)

ax.set_title("Test data, 1D interpolations, and ground truth")

ax.plot(x_range, y_truth, c="green", label="ground truth")
ax.plot(x_range, y_linear, ls=":", label="linear")
ax.plot(x_range, y_cubic, ls="--", label="cubic")
ax.scatter(x_samples, y_samples, c="black", label="data")

ax.legend()

plt.show()

In [ ]:
# computing derivative (as function)
lin_deriv = lin_interp.derivative()
cubic_deriv = cubic_interp.derivative()

# evaluating at the plot points
y_truth_der = magic.interp_1d_truth_der(x_range)
y_lin_der = lin_deriv(x_range)
y_cubic_der = cubic_deriv(x_range)

# ploting the derivatives

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlim(0, 10)

ax.set_title("Test data, 1D interpolations, and ground truth")

ax.plot(x_range, y_truth_der, c="green", label="ground truth")
ax.plot(x_range, y_lin_der, ls=":", label="linear")
ax.plot(x_range, y_cubic_der, ls="--", label="cubic")

ax.legend()

plt.show()

In [ ]:
# computing definite integral

cubic_intg = cubic_interp.integral(x_samples[0], x_samples[-1])
print(f"Integral (cubic interpolation): {cubic_intg:.5f}")

### Univariate spline curve fitting

In [ ]:
# loading test data
x_samples = magic.fit_1d_samp_x.copy()
y_samples = magic.fit_1d_samp_y.copy()

In [ ]:
# plot the test data

fig = plt.figure()
ax = fig.add_subplot()

ax.scatter(x_samples, y_samples, c="k", marker="x")

plt.show()

In [ ]:
# trying different uncertainty (0.1, 0.2, 0.5) when fitting

cubic_fit_under = sint.UnivariateSpline(
    x_samples, y_samples, 
    w = np.full_like(x_samples, 1/0.5), k = 3, s = len(x_samples)
)
cubic_fit_mid   = sint.UnivariateSpline(
    x_samples, y_samples, 
    w = np.full_like(x_samples, 1/0.2), k = 3, s = len(x_samples)
)
cubic_fit_over  = sint.UnivariateSpline(
    x_samples, y_samples, 
    w = np.full_like(x_samples, 1/0.1), k = 3, s = len(x_samples)
)

In [ ]:
# compute x and y values for fitted spline
x_range = np.linspace(0, 10, 201)
y_cubic_under = cubic_fit_under(x_range)
y_cubic_mid = cubic_fit_mid(x_range)
y_cubic_over = cubic_fit_over(x_range)

In [ ]:
# plot the test data with fit

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(x_range, y_cubic_mid, ls="-.", label="balanced fit")
ax.plot(x_range, y_cubic_under, ls=":", label="underfit")
ax.plot(x_range, y_cubic_over, ls="--", label="overfit")

ax.scatter(x_samples, y_samples, c="k", marker="x")

ax.legend()

plt.show()

In [ ]:
# check with ground truth
y_truth = magic.fit_1d_truth(x_range)

In [ ]:
# plot the test data, fit, and ground truth

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(x_range, y_cubic_mid, ls="-.", label="balanced fit")
ax.plot(x_range, y_cubic_under, ls=":", label="underfit")
ax.plot(x_range, y_cubic_over, ls="--", label="overfit")
ax.plot(x_range, y_truth, ls="-", label="ground truth")

ax.scatter(x_samples, y_samples, c="k", marker="x")

ax.legend()

plt.show()

In [ ]:
# determine the best smoothness parameter self-consistently

cubic_fit_auto = sint.make_smoothing_spline(x_samples, y_samples)
y_cubic_auto = cubic_fit_auto(x_range)

In [ ]:
# plot the test data with the self-consistent fit

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(x_range, y_cubic_auto, ls="-.", label="auto fit")
ax.plot(x_range, y_truth, ls="-", color="tab:red", label="ground truth")

ax.scatter(x_samples, y_samples, c="k", marker="x")

ax.legend()

plt.show()

In [ ]:
# function to compute residual for output of make_smoothing_spline()

def get_residual(x, y, spl, w=None):
    '''
    get the residual (weighed sum of |y - spl(x)|**2) of the smoothing spline

    arguments:
      x: a numpy array of independent variables
      y: a numpy array (shape matching x) of dependent varibles
      w: the weight vector (shape matching x). 
        If None defaults to np.ones_like(x)
      spl: the BSpline object representing data fit

    output:
      the weighted sum of residuals (sum is over axis zero).
    '''
    if w is None:
        w = np.ones_like(x)
    
    return np.sum(w * (y - spl(x))**2, axis=0)

In [ ]:
# calculate the total residual

auto_residual = get_residual(x_samples, y_samples, cubic_fit_auto)
print(auto_residual)

In [ ]:
# calculate the uncertainty

auto_std = np.sqrt(auto_residual / len(x_samples))
print(auto_std)

### Weighed univariate spline curve fitting

In [ ]:
# dataset with non-uniform error bar
x_samples = magic.fit_1d_w_samp_x.copy()
y_samples = magic.fit_1d_w_samp_y.copy()
y_errors = magic.fit_1d_weights.copy()

In [ ]:
# plot the test data with fit

fig = plt.figure()
ax = fig.add_subplot()

ax.errorbar(
    x_samples, y_samples, y_errors, 
    color="k", ls="none", marker="x", 
    capsize=4, ecolor="gray", label="data"
)

ax.legend()

plt.show()

In [ ]:
# determine best-fitted UNWEIGHED spline
y_errors_avg = np.mean(y_errors)
cubic_fit_unweighed = sint.UnivariateSpline(
    x_samples, y_samples, w=np.full_like(x_samples, 1/y_errors_avg), k=3, s=len(x_samples)
)

# create x and y values at plot points
x_range = np.linspace(0, 10, 201)
y_cubic_unweighed = cubic_fit_unweighed(x_range)

In [ ]:
# determine best-fitted WEIGHED spline
cubic_fit_weighed = sint.UnivariateSpline(x_samples, y_samples, w=1/y_errors, k=3, s=len(x_samples))

# create x and y values at plot points
x_range = np.linspace(0, 10, 201)
y_cubic_weighed = cubic_fit_weighed(x_range)

In [ ]:
# plot the test data with fit

fig = plt.figure()
ax = fig.add_subplot()

ax.errorbar(
    x_samples, y_samples, y_errors, 
    color="k", ls="none", marker="x", 
    capsize=4, ecolor="gray", label="data"
)
ax.plot(x_range, y_cubic_unweighed, ls="-.", c="tab:blue", label="unweighed")
ax.plot(x_range, y_cubic_weighed, ls="--", c="tab:red", label="weighed")

ax.legend()

plt.show()

### Univariate fit to theoretical model

In [ ]:
# loading test data
x_samples = magic.theory_1d_samp_x.copy()
y_samples = magic.theory_1d_samp_y.copy()

In [ ]:
# plot the test data

fig = plt.figure()
ax = fig.add_subplot()

ax.scatter(x_samples, y_samples, c="k", marker="x")

plt.show()

In [ ]:
# define a "theoretical curve" with parameters a, b, c to be determined

def f_theory(x, a, b, c):
    return a + b * np.exp(-c * x)

In [ ]:
# find the paramters for best-fitted curve
out = curve_fit(f_theory, x_samples, y_samples)
out

In [ ]:
# evaluate the fitted value on plot points

x_range = np.linspace(0, 10, 201)
y_fitted = f_theory(x_range, out[0][0], out[0][1], out[0][2])

In [ ]:
# plot the theory fit against data

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(x_range, y_fitted, ls="-.", label="fit")
ax.scatter(x_samples, y_samples, c="k", marker="x", label="data")

ax.legend()

plt.show()

In [ ]:
# get the ground truth

y_truth = magic.theory_1d_truth(x_range)

# plot data, fit, and ground truth

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(x_range, y_fitted, ls="-.", label="fit")
ax.plot(x_range, y_truth, ls="-", c="tab:red", label="ground truth")
ax.scatter(x_samples, y_samples, c="k", marker="x", label="data")

ax.legend()

plt.show()

In [ ]:
# show the estimated parameters again
out[0]

In [ ]:
# check with ground truth
magic.theory_1d_params_truth

In [ ]:
# show the estimated std in parameters
params_std = np.sqrt(np.diag(out[1]))
params_std

In [ ]:
# we expect the deviation from ground thuth to be approximately 1 sd
(out[0] - np.array(magic.theory_1d_params_truth)) / params_std

In [ ]:
# full output mode
full_out = curve_fit(f_theory, x_samples, y_samples, full_output=True)
full_out

In [ ]:
# compute residual
residual = np.sum(full_out[2]["fvec"] **2)

print(residual)

# compute estimated standard deviation
std = np.sqrt(residual / len(x_samples))

print(std)

In [ ]:
# check with ground truth
magic.theory_1d_noise_sd

### Exercise 1

_**Code writing #1.**_ Corrupted ERIS data

The file ERIS_corrupted.csv on the data folder contains data collected by the CTD instrument from the UW ERIS program. About 15% of the data is missing.* We’ll focus on turbidity and use interpolation to fill in the missing data. The procedure is as follows:

1. Separate the data into two parts: those that are missing and those that are present (hint: use `pd.isna()` and `pd.notna()`)
2. Create the spline function from the data present (hint: use `.astype("int64")` to convert date to numerical values)
3. Use the spline function to fill in turbidity values for the dates with missing data
4. Combine the data present and the interpolated data, and plot the results (hint: use `pd.concat()`)

In [ ]:
# starter code: load the "corrupted" ERIS data

eris = pd.read_csv("data/ERIS_corrupted.csv").convert_dtypes()
eris["date"] = pd.to_datetime(eris["date"])

In [ ]:
# starter code: plot turbidity from the corrupted data

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(eris["date"], eris["turbidity"], color="tab:blue")
ax.tick_params("x", rotation=-45)

plt.show(fig)

In [ ]:
# step 1: seperate the data into two parts

In [ ]:
# step 2: construct spline from data that aren't NAs

In [ ]:
# step 3: filling in the turbidity values for dates with NAs

In [ ]:
# step 4: recombine the two parts and plot the result again

In [ ]:
# starter code: plotting the repaired turbidity data

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(eris_repaired["date"], eris_repaired["turbidity"], ls="--", color="tab:red", label="interpolated")
ax.plot(eris["date"], eris["turbidity"], color="tab:blue", label="original data")

ax.tick_params("x", rotation=-75)
ax.legend()

plt.show(fig)

### Exercise 2

_**Code Writing #2.**_ Keeling curve

The file monthly_in_situ_co2_mlo.csv contains the famous Keeling curve data. Focus on the raw observational data (column CO2) versus date and fit the result to two possible model

a. $\textrm{[CO$_2$]} = a + b \ (\textrm{Year} − 1970)$<br>
b. $\textrm{[CO$_2$]} = a + b \ (\textrm{Year} − 1970) + c \ (\textrm{Year} − 1970)^2$


We’ll proceed as follows:

1. Define two functions that corresponds to the two model
1. Use `curve_fit()` to find the best-fit parameters of the two models, and their uncertainties
1. Plot the two best-fit curves against observation data
1. Comment on whether the increase in atmospheric carbon is _accelerating_ or not

In [ ]:
# starter code: load the keeling curve data
keeling = pd.read_csv(
    "data/monthly_in_situ_co2_mlo.csv",
    names = [ # specifying column name to avoid issues
        "year", "month", "date_excel", "date", 
        "CO2", "CO2_adj", "CO2_fitted", "CO2_adj_fitted",
        "CO2_filled", "CO2_adj_filled", "station"
    ], skiprows=64
).convert_dtypes()

# skip to largest block without NAs
keeling = keeling[
    np.logical_and(keeling["year"] > 1964, keeling["year"] < 2025)
]

In [ ]:
# starter code: plot the raw data

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(keeling["date"], keeling["CO2"])

ax.set_xlabel("year")
ax.set_ylabel("atmospheric CO$_2$ (ppm)")

plt.show(fig)

In [ ]:
# step 1: define two functions for the two models

In [ ]:
# step 2: perform curve fit, extract parameters and their uncertainties

In [ ]:
# step 3: plot the best-fit curves against obvervation data

In [ ]:
# step 4: comment on acceleration

## Interpolation and curve fitting in higher dimensions

### Interpolating 2D temperature grid

In [ ]:
# load temperature meaurement on regular grid

x_loc = magic.x_temp_loc.copy()
y_loc = magic.y_temp_loc.copy()
x_grid, y_grid = np.meshgrid(x_loc, y_loc)
temp_grid = magic.temp_grid.copy()

In [ ]:
# check the shape of data

print(x_loc.shape, y_loc.shape, x_grid.shape, y_grid.shape, temp_grid.shape)

In [ ]:
# load the path on which we need to interpolate

t_path = magic.t_path.copy()
x_path = magic.x_path.copy()
y_path = magic.y_path.copy()

In [ ]:
# check the shape of the path variables

print(t_path.shape, x_path.shape, y_path.shape)

In [ ]:
# plot the temperature measurement and the path to tranverse

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot()

ax.set_xticks(np.arange(0, 3.1, 0.5))
ax.set_yticks(np.arange(0, 2.1, 0.5))
ax.set_aspect("equal")

s = ax.scatter(x_grid.flatten(), y_grid.flatten(), c=temp_grid.flatten(), cmap="plasma")
plt.plot(x_path, y_path, color="k", marker="x")

ax.text(x_path[0] + 0.05, y_path[0], "start")
ax.text(x_path[-1] - 0.25, y_path[-1] - 0.05, "end")

cb = fig.colorbar(s)

cb.set_label("Temperature (°C)")

plt.show()

In [ ]:
# geneating the spline function
# note that the dimension convention for z is flippped

temp_interp = sint.RectBivariateSpline(
    x_loc, y_loc, temp_grid.transpose(), kx=3, ky=3, s=0
)

In [ ]:
# create array of temperture along the path
temp_path = temp_interp(x_path, y_path, grid=False)

In [ ]:
# plot temperature as function of time

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlabel("Time (minutes)")
ax.set_ylabel("Temperature (°C)")

plt.plot(t_path, temp_path, color="k", marker="x")

plt.show()

### Smoothing spline on regular 2D grid

In [ ]:
# load the noisy data
X_coords = magic.X_noisy.copy()
Y_coords = magic.Y_noisy.copy()
Z_noisy = magic.Z_noisy.copy()

In [ ]:
# plot noisy data

fig = plt.figure(figsize=(7,4))
ax = fig.add_subplot()
ax.set_aspect(1)

mesh = ax.pcolormesh(X_coords, Y_coords, Z_noisy, vmin=-5, vmax=5)
cb = fig.colorbar(mesh)

plt.show()

In [ ]:
# Use smoothing spline as an attempt to recover underlying clean data
# again note that the dimension convention for z is flippped
Z_interp_func = sint.RectBivariateSpline(
    X_coords, Y_coords, Z_noisy.transpose(), s = 0.3**2 * Z_noisy.size
)

In [ ]:
# calculate the smoothened z-value on the grid
Z_interp = Z_interp_func(X_coords, Y_coords, grid=True).transpose()

In [ ]:
# plot the smoothened data

fig = plt.figure(figsize=(7,4))
ax = fig.add_subplot()
ax.set_aspect(1)

mesh = ax.pcolormesh(X_coords, Y_coords, Z_interp, vmin=-5, vmax=5)
cb = fig.colorbar(mesh)

plt.show()

In [ ]:
# compare with ground truth
Z_clean = magic.Z_clean.copy()

fig = plt.figure(figsize=(7,4))
ax = fig.add_subplot()
ax.set_aspect(1)

mesh = ax.pcolormesh(X_coords, Y_coords, Z_clean, vmin=-5, vmax=5)
cb = fig.colorbar(mesh)

plt.show()

### Interpolating non-gridded 2D data

In [ ]:
# loading ungridded data of NO3- concentration
x_stations = magic.x_stations.copy()
y_stations = magic.y_stations.copy()
no3_stations = magic.NO3_stations.copy()

In [ ]:
# check the dimension of data
print(x_stations.shape, y_stations.shape, no3_stations.shape)

In [ ]:
# plot the measured data

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot()

ax.set_xticks(np.arange(0, 31, 5))
ax.set_yticks(np.arange(0, 21, 5))
ax.set_aspect("equal")

s = ax.scatter(
    x_stations, y_stations, c=no3_stations, 
    cmap="plasma", vmin=0, vmax=7
)
cb = fig.colorbar(s)

cb.set_label("Temperature (°C)")

plt.show()

In [ ]:
# make cubic and linear interpolation

no3_interp = sint.CloughTocher2DInterpolator(
    np.vstack([x_stations, y_stations]).transpose(), no3_stations)
no3_lin_interp = sint.LinearNDInterpolator(
    np.vstack([x_stations, y_stations]).transpose(), no3_stations)

In [ ]:
# get interpolated value on a grid

x_loc = np.linspace(0, 30, 91)
y_loc = np.linspace(0, 20, 61)
x_grid, y_grid = np.meshgrid(x_loc, y_loc)
no3_grid = no3_interp(x_grid, y_grid)
no3_lin_grid = no3_lin_interp(x_grid, y_grid)

In [ ]:
# plot the result from cubic interpolation

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot()

ax.set_xticks(np.arange(0, 31, 5))
ax.set_yticks(np.arange(0, 21, 5))
ax.set_aspect("equal")

mesh = ax.pcolormesh(
    x_loc, y_loc, no3_grid,
    cmap="plasma", vmin=0, vmax=7
)

pt = ax.scatter(x_stations, y_stations, c="k")

cb = fig.colorbar(mesh)
cb.set_label("Temperature (°C)")

plt.show()

In [ ]:
# plot the result from linear interpolation

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot()

ax.set_xticks(np.arange(0, 31, 5))
ax.set_yticks(np.arange(0, 21, 5))
ax.set_aspect("equal")

mesh = ax.pcolormesh(
    x_loc, y_loc, no3_lin_grid,
    cmap="plasma", vmin=0, vmax=7
)

pt = ax.scatter(x_stations, y_stations, c="k")

cb = fig.colorbar(mesh)
cb.set_label("Temperature (°C)")

plt.show()

In [ ]:
# compare with ground truth

# loading ground-truth data
no3_truth = magic.NO3_2d_truth(x_grid, y_grid)

# plot ground-truth data
fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot()

ax.set_xticks(np.arange(0, 31, 5))
ax.set_yticks(np.arange(0, 21, 5))
ax.set_aspect("equal")

mesh = ax.pcolormesh(
    x_loc, y_loc, no3_truth,
    cmap="plasma", vmin=0, vmax=7
)

pt = ax.scatter(x_stations, y_stations, c="k")

cb = fig.colorbar(mesh)
cb.set_label("Temperature (°C)")

plt.show()

### Exercise 3

In [ ]:
# starter code: load the data set

data = xr.open_dataset("data/mercatorbiomer4v2r1_AmPac_mean_nut_20250131.nc")
display(data)

In [ ]:
# starter code: get the paths
probe_hour = magic.probe_hour.copy()
probe_lat = magic.probe_lat.copy()
probe_lon = magic.probe_lon.copy()
probe_depth = magic.probe_depth.copy()

In [ ]:
# start code: plot the path of the probe

fig = plt.figure(figsize=(9, 4))
ax1 = fig.add_subplot(1, 5, (1, 4), projection=ccrs.PlateCarree())
ax2 = fig.add_subplot(1, 5, 5)

ax1.set_extent([-150, -120, 30, 50], crs=ccrs.PlateCarree())
ax1.coastlines()
ax1.add_feature(cfeature.OCEAN, color="skyblue")
ax1.plot(probe_lon, probe_lat, c="k", marker="x")

ax1.text(probe_lon[0]+0.2, probe_lat[0]+0.2, "start")
ax1.text(probe_lon[-1]+0.4, probe_lat[-1]-0.4, "end")

gl = ax1.gridlines(
    crs=ccrs.PlateCarree(), draw_labels=True,
    linewidth=1, color='darkgray', linestyle='--'
)
gl.left_labels = True
gl.right_labels = False
gl.top_labels = True
gl.bottom_labels = False

ax2.plot(probe_depth, probe_hour, c="k", marker="x")
ax2.set_xlim(0, 110)
ax2.invert_xaxis()
ax2.set_xticks(np.arange(0, 101, 20))
ax2.set_xlabel("depth (m)")
ax2.set_ylim(0, 24)
ax2.invert_yaxis()
ax2.set_yticks(np.arange(0, 24.5, 6))
ax2.set_ylabel("hour")

plt.show(fig)

In [ ]:
# part 1: subset the data to a grid that contains no nan's

In [ ]:
# part 2: perform cubic interpolation to obtain function-like object

In [ ]:
# part 3: evaluate the interpolated function at path points
# call the result probe_po4

probe_po4 = ...

In [ ]:
# starter code: plot the result

fig = plt.figure()
ax = fig.add_subplot()

plt.plot(probe_hour, probe_po4, ls="--", color="k", marker="x")
ax.set_ylabel("phosphate concentration (mmol/m$^3$)")
ax.set_xlim(0, 24)
ax.set_xticks(np.arange(0, 24.5, 6))
ax.set_xlabel("hour")

plt.show(fig)

### Exercise 4

In [ ]:
# starter code: loading data

probe_x = magic.probe_x.copy()
probe_y = magic.probe_y.copy()
probe_t = magic.probe_t.copy()
probe_sal = magic.probe_sal.copy()

In [ ]:
# starter code: plot the initial path
# and salinity measurement along path

fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(6, 10))

ax1.set_xlim(-10, 10)
ax1.set_ylim(-10, 10)

ax1.set_aspect(1)
ax1.plot(probe_x, probe_y, "k-x")

ax1.text(probe_x[0] - 2, probe_y[0] - 0.5, "start")
ax1.text(probe_x[-1] + 0.2, probe_y[-1] + 0.3, "end")

ax2.plot(probe_t, probe_sal, "k--x")

plt.show(fig)

In [ ]:
# starter code: plot salinity value at measurement location

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax.set_aspect(1)
sc = ax.scatter(probe_x, probe_y, c=probe_sal, vmin=3, vmax=4)
cb = fig.colorbar(sc)


plt.show(fig)

In [ ]:
# part 1: create smoothing spline

In [ ]:
# part 2: evaluate smoothed value on a grid
# use the x-coordinates and y-coordinates provided
# and call the result sal_grid

x_coords = np.linspace(-10, 10, 51)
y_coords = np.linspace(-10, 10, 51)

sal_grid = ...

In [ ]:
# starter code: plot the result

fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
ax.set_aspect(1)

mesh = ax.pcolormesh(x_coords, y_coords, sal_grid, vmin=3, vmax=4)
cb = fig.colorbar(mesh)

ax2.plot(probe_t, probe_sal, "k--x")

plt.show(fig)